In [9]:
import pandas as pd

df_2022 = pd.read_csv('atp_matches_2022.csv')
df_2023 = pd.read_csv('atp_matches_2023.csv')

df = pd.concat([df_2022, df_2023], ignore_index=True)

In [10]:
wimbledon = df[df['tourney_name'].str.contains("Wimbledon", case=False)]


In [11]:
wimbledon['rank_diff'] = wimbledon['winner_rank'] - wimbledon['loser_rank']

/var/folders/6y/x3xngpjd5z75w65f0fk_lgh40000gn/T/ipykernel_5580/713175086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wimbledon['rank_diff'] = wimbledon['winner_rank'] - wimbledon['loser_rank']


In [12]:
X = wimbledon[['rank_diff', 'best_of']]
y = [1] * len(wimbledon)  # winner (we’ll later format matches as pair-based)


In [16]:
from xgboost import XGBClassifier

X = wimbledon[['rank_diff', 'best_of']]
y = [1] * len(wimbledon)  # winner (we’ll later format matches as pair-based)

model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8
)

model.fit(X, y)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]

In [17]:
import pandas as pd
from xgboost import XGBClassifier

# Load data
df_2022 = pd.read_csv("atp_matches_2022.csv")
df_2023 = pd.read_csv("atp_matches_2023.csv")
df = pd.concat([df_2022, df_2023], ignore_index=True)

# Filter for Wimbledon only
wimbledon = df[df['tourney_name'].str.contains("Wimbledon", case=False)].copy()

# Build winner-side records
winner_data = pd.DataFrame({
    'player_rank': wimbledon['winner_rank'],
    'opponent_rank': wimbledon['loser_rank'],
    'best_of': wimbledon['best_of'],
    'label': 1
})

# Build loser-side records
loser_data = pd.DataFrame({
    'player_rank': wimbledon['loser_rank'],
    'opponent_rank': wimbledon['winner_rank'],
    'best_of': wimbledon['best_of'],
    'label': 0
})

# Combine both
data = pd.concat([winner_data, loser_data], ignore_index=True)

# Feature: rank difference (smaller rank number = better player)
data['rank_diff'] = data['opponent_rank'] - data['player_rank']

# Drop rows with missing ranks
data = data.dropna(subset=['player_rank', 'opponent_rank'])

# Define X and y
X = data[['player_rank', 'opponent_rank', 'rank_diff', 'best_of']]
y = data['label']

# Train XGBoost model
model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X, y)

print("✅ Model trained successfully on Wimbledon matches!")


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [20:28:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Model trained successfully on Wimbledon matches!
